## Basic demonstration of sansmic
The following examples show a withdrawal of 1 MMbbl from an approx. 10 MMbbl cavern over 10 days, followed by a 45 day rest period. The first example shows how to load and run from an existing DAT-format file. The second example shows how to build the same scenario from scratch.

### Setup
After installing sansmic, import the ``sansmic`` module. If you need other packages, import them as well.

In [ ]:
import sansmic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Using an existing SANSMIC DAT file

If you have an existing file, such as the provided example called ``old.dat``, then you can just read it in to create a new scenario object. You can look at the object in dictionary format to see how it was imported.

In [ ]:
# Read in the old .DAT file
test1scen = sansmic.io.read_scenario('old.dat')

# write a new-style TOML file and print it to check it out
sansmic.io.write_scenario(test1scen, 'converted.toml')
with open('converted.toml', 'r') as fin:
    for line in fin.readlines():
        print(line.strip())

To run the simulation in batch mode, simply create a new simulation and then run it. The results are stored in the ``results`` attribute of the simulation object.

In [ ]:
with test1scen.new_simulation('converted') as sim1:
    sim1.run_sim()
test1results = sim1.results

In [ ]:
test1results.df_t_1D.V_cav.plot()

In [ ]:
test2scen = sansmic.Scenario(
    title="Simple example",
    cavern_height=2000.0,  # z-domain = [0, 1000] ft
    floor_depth=4000.0,    # TD = 4000 ft MD
    num_cells=200,         # 200 cells (10 ft high)
)
test2scen.insolubles_ratio = 0.04
test2scen.geometry_format = sansmic.model.GeometryFormat.RADIUS_LIST
radii = np.array([100]*201)  # 100 ft radius for the bulk of the cavern
radii[0] = 50
radii[1] = 90
radii[187] = 88
radii[188] = 45
radii[189] = 15
radii[190] = 8
radii[191] = 5
radii[192:] = 2


Having the geometry data in the data structure makes it cumbersome to view it. So let's save the data in a file instead.

In [ ]:
# If you wanted to keep the data directly with the configuration, 
# you would uncomment the line below and comment out the rest.
#
# test2scen.geometry_data = dict(radii=radii.tolist())

with open('scratch.geom', 'w') as fout:
    for v in radii:
        fout.write('{}\n'.format(v))
test2scen.geometry_data = "scratch.geom"

Now we add a leaching stage. Remeber that when creating a new stage it automatically adds it also.

In [ ]:
stage1 = test2scen.new_stage()

Now we setup up the simulation stage. Note - in this example we will set up a timestep that is ten times larger (0.1 h) than the old data file so that we can see the differences, if any.

In [ ]:
stage1.title = "Found a bug - have to have a title?"
stage1.simulation_mode = 'withdrawal'
stage1.brine_injection_sg = 1.003     # sg
stage1.brine_injection_rate = 100000  # bbl/d
stage1.brine_injection_depth = 15  # ft MD
stage1.brine_interface_depth = 37   # ft MD
stage1.injection_duration = 360       # h
stage1.rest_duration = 2520           # h
stage1.inner_tbg_inside_diam = 9.85    # in
stage1.inner_tbg_outside_diam = 10.75 # in
stage1.outer_csg_inside_diam = 9.85  # in
stage1.outer_csg_outside_diam = 10.75 # in
stage1.set_cavern_sg = 1.2019         # starting cavern SG
stage1.solver_timestep = 0.1          # h
stage1.save_frequency = 120           # timesteps (120 ts x 0.1 h/ts = 12 h)

Let's save this new scenario in a TOML file, and then read it back in to see how it was formatted.

In [ ]:
sansmic.io.write_scenario(test2scen, 'scratch.toml')
with open('scratch.toml', 'r') as fin:
    for line in fin.readlines():
        print(line.strip())

Now let's run this simulation through a python iterator. This allows us to pull out results at any time we want to request them by using the ``get_current_state`` function.

In [ ]:
with test2scen.new_simulation('scratch', verbosity=1) as sim2:
    print("""         t_d        V_inj        V_cav       sg_ave""")
    print("""      ------  -----------  -----------  -----------""")
    for stage, step in sim2.steps:
        if step % 2400 == 0:
            res = sim2.get_current_state()
            print(res.df_t_1D.loc[:,['t_d', 'V_inj', 'V_cav', 'sg_ave']].to_string(header=False, float_format='%12.4g', index=False))
test2results = sim2.results

In [ ]:
test2results.df_t_1D.V_cav.plot()

In [ ]:
ax1 = test1results.df_t_1D.plot(y='sg_ave', x='t_d', label='Test 1 sg ave', marker='.')
ax2 = test1results.df_t_1D.plot(y='err_ode', x='t_d', label='Test 1 ODE err', marker='.')
test2results.df_t_1D.plot(ax=ax1, y='sg_ave', x='t_d', label='Test 2 sg ave', marker='.')
test2results.df_t_1D.plot(ax=ax2, y='err_ode', x='t_d', label='Test 2 ODE err', marker='.')

## Running from the command line
Jupyter is not a great way to demonstrate how to run sansmic from the command line; however, it is still possible to do so. The following code simulates the execution of the following command:

``sansmic old.dat -o cmdlineTest --no-json --no-hdf``

Try adding "-v" or "-vv" at the end of the list to see the difference.

In [ ]:
import sansmic.app
resultsCmdLine = sansmic.app.main(['old.dat','-o','cmdlineTest','--no-json','--no-hdf',], ret=True)

In [ ]:
resultsCmdLine.df_t_1D.loc[[0,len(resultsCmdLine.time)-1]].to_dict()

## Multi-stage model
Now, let's try running a model with several leaching stages. Consider the following scenario:

A site is delivering 5 MMbbl of oil as ten 500 Mbbl cargos (taking 20 hours to deliver each cargo). There are two days of downtime between deliveries. 
The stream is blended between three different caverns, and our cavern of interest is only responsible for delivering 7500 bbl/h. The starting geometry
will be the one we put in "scratch.geom", earlier. At the end, we will give the cavern an additional 90 days to equilibrate.

In [ ]:
tenCargos = sansmic.Scenario(
    title="Simple example",
    cavern_height=2000.0,  # z-domain = [0, 1000] ft
    floor_depth=4000.0,  # TD = 4000 ft MD
    num_cells=200,  # 200 cells (10 ft high)
)
tenCargos.insolubles_ratio = 0.04
tenCargos.geometry_format = sansmic.model.GeometryFormat.RADIUS_LIST
tenCargos.geometry_data = "scratch.geom"

# Set some of these as default values for each stage
tenCargos.defaults = dict(
    inner_tbg_inside_diam = 9.85, # in
    inner_tbg_outside_diam = 10.75,  # in
    outer_csg_inside_diam = 9.85,  # in
    outer_csg_outside_diam = 10.75,  # in
    solver_timestep = 0.1, # h
    save_frequency = 10,  # save every hour
)

for i in range(10):
    stage = tenCargos.new_stage()
    stage.title = "Cargo number {}".format(i)
    stage.simulation_mode = "withdrawal"
    stage.brine_injection_sg = 1.003  # sg
    stage.brine_injection_rate = 7500 * 24  # bbl/h * 24 h/d
    stage.brine_injection_depth = 15
    stage.injection_duration = 20  # h
    stage.rest_duration = 4 + 24 * 2  # two days downtime plus the four hours
    if i == 0:
        # We only want to set these on the first stage, after that we want
        # to use what was calculated previously (i.e., leave the values None)
        stage.brine_interface_depth = 37 # ft MD starting
        stage.set_cavern_sg = 1.2019  # starting cavern SG
    if i == 9:
        # add an extra 60 days to the end
        stage.rest_duration = stage.rest_duration + 90 * 24

sansmic.io.write_scenario(tenCargos, "tenCargos.toml")

# run the model
with tenCargos.new_simulation("tenCargos", verbosity=0) as sim3:
    sim3.run_sim()
tenCargoResults = sim3.results

In [ ]:
# plot some overview statistics for performance analysis
ax0 = tenCargoResults.df_t_1D.plot(y='V_cav', x='t_d', label='Cavern volume', marker='.')
ax1 = tenCargoResults.df_t_1D.plot(y='sg_ave', x='t_d', label='Average cavern sg', marker='.')
ax2 = tenCargoResults.df_t_1D.plot(y='err_ode', x='t_d', label='ODE err', marker='.')

### Comparing the ten-cargo simulation to the all-at-once simulation
You may have noticed that all of the different simulations have resulted in a 1.5 million barrel withdrawal from the cavern. Now we can compare the results of splitting the injection into ten cargos compared to the results if all 1.5 million barrels of raw water were injected all at once. First, we can look at the final results in a table.

In [ ]:
pd.DataFrame.from_dict({'single-inject': resultsCmdLine.df_t_1D.iloc[-1,:], 'ten-cargos': tenCargoResults.df_t_1D.iloc[-1,:], 'diff': resultsCmdLine.df_t_1D.iloc[-1,:] - tenCargoResults.df_t_1D.iloc[-1,:]})

The final results look very close. Most of the differences are much less than the absolute values except for the jet length, which *should* be different since there are vastly different velocities involved. So, how does the timeseries of changes look?

In [ ]:
ax = resultsCmdLine.df_t_1D.plot(x='t_d', y='sg_ave', label='single-inject')
tenCargoResults.df_t_1D.plot(ax=ax, x='t_d', y='sg_ave', label='ten-cargo')
ax.set_ylabel('Cavern average sg')
ax.set_xlabel('Time (d)')
ax = resultsCmdLine.df_t_1D.plot(x='t_d', y='V_cav', label='single-inject')
tenCargoResults.df_t_1D.plot(ax=ax, x='t_d', y='V_cav', label='ten-cargo')
ax.set_ylabel('Cavern volume (bbl)')
ax.set_xlabel('Time (d)')

Finally, let's take a look at the final cavern shape. We can do this by plotting the depths against the radius for the last time period.

In [ ]:
plt.plot( resultsCmdLine.radius.iloc[:,-1],-resultsCmdLine.depths, label='single-inject')
plt.plot( tenCargoResults.radius.iloc[:,-1],-tenCargoResults.depths, label='ten-cargo')
plt.legend()

These shapes are similar, but as expected, the ten-cargo method has a different slope since it has time to leach between stages. We can see this better by zooming in.

In [ ]:
plt.plot( resultsCmdLine.radius.iloc[:,-1],-resultsCmdLine.depths, label='single-inject')
plt.plot( tenCargoResults.radius.iloc[:,-1],-tenCargoResults.depths, label='ten-cargo')
plt.legend()
plt.ylim(-4000, -3500)
plt.xlim(90,120)

Hopefully this has been a useful introduction to using the new version of sansmic! There will be an additional notebook that compares the old FORTRAN code results with the new results soon.